<a href="https://colab.research.google.com/github/KJahnavi-1/Smart-Sorting-Transfer-Learning-for-Identifying-Rotten-Fruits-and-Vegetables/blob/main/SmartSorting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from keras.optimizers import Adam
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np


In [ ]:
!unzip '/content/drive/MyDrive/archive (1).zip'

Archive:  /content/drive/MyDrive/archive (1).zip
replace Fruit And Vegetable Diseases Dataset/Apple__Healthy/FreshApple (1).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
dataset_dir = '/content/Fruit And Vegetable Diseases Dataset'